# Creating and Valuing a CDS Contract

Creates a CDS contract and does a valuation and calculates risk measures

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.33 - This build:  11 Nov 2023 at 07:47 #
#     This software is distributed FREE AND WITHOUT ANY WARRANTY   #
#  Report bugs as issues at https://github.com/domokane/FinancePy  #
####################################################################



## Creating a CDS Contract

In [3]:
value_date = Date(9, 8, 2019)
effective_date = value_date
maturity_date = Date(20, 6, 2029)
cdsCoupon = 0.0150
notional = ONE_MILLION
long_protection = False
tradeDate = Date(9, 8, 2019)

In [4]:
cds_contract = CDS(effective_date, maturity_date, cdsCoupon, notional, long_protection)

## Build Ibor Curve

In [5]:
settle_date = effective_date

In [6]:
dcType = DayCountTypes.ACT_360
depo1 = IborDeposit(settle_date, "1M", 0.022009, dcType)
depo2 = IborDeposit(settle_date, "2M", 0.022138, dcType)
depo3 = IborDeposit(settle_date, "3M", 0.021810, dcType)
depo4 = IborDeposit(settle_date, "6M", 0.020503, dcType)
depo5 = IborDeposit(settle_date, "12M", 0.019930, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swapType = SwapTypes.PAY

swap1 = IborSwap(settle_date,"2Y",swapType,0.015910,fixedFreq,dcType)
swap2 = IborSwap(settle_date,"3Y",swapType,0.014990,fixedFreq,dcType)
swap3 = IborSwap(settle_date,"4Y",swapType,0.014725,fixedFreq,dcType)
swap4 = IborSwap(settle_date,"5Y",swapType,0.014640,fixedFreq,dcType)
swap5 = IborSwap(settle_date,"6Y",swapType,0.014800,fixedFreq,dcType)
swap6 = IborSwap(settle_date,"7Y",swapType,0.014995,fixedFreq,dcType)
swap7 = IborSwap(settle_date,"8Y",swapType,0.015180,fixedFreq,dcType)
swap8 = IborSwap(settle_date,"9Y",swapType,0.015610,fixedFreq,dcType)
swap9 = IborSwap(settle_date,"10Y",swapType,0.0159880,fixedFreq,dcType)
swap10 = IborSwap(settle_date,"12Y",swapType,0.016430,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

libor_curve = IborSingleCurve(value_date, depos, [], swaps)

# Build a CDS Curve

In [7]:
cds1 = CDS(settle_date, "1Y", 0.0200)
cds2 = CDS(settle_date, "2Y", 0.0220)
cds3 = CDS(settle_date, "3Y", 0.0250)
cds4 = CDS(settle_date, "4Y", 0.0275)
cds5 = CDS(settle_date, "5Y", 0.0290)
cds6 = CDS(settle_date, "7Y", 0.0300)
cds7 = CDS(settle_date, "10Y", 0.0310)
cds8 = CDS(settle_date, "15Y", 0.0315)

In [8]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [9]:
recovery_rate = 0.40

In [10]:
issuer_curve = CDSCurve(value_date, cdss, libor_curve, recovery_rate)

In [11]:
print(issuer_curve)

OBJECT TYPE: CDSCurve
TIME,SURVIVAL_PROBABILITY
 0.0000000,  1.0000000
 1.1178082,  0.9630225
 2.1178082,  0.9242448
 3.1178082,  0.8757579
 4.1178082,  0.8236065
 5.1205479,  0.7744884
 7.1205479,  0.6922011
10.1232877,  0.5814139
15.1260274,  0.4390083


# Valuation

In [12]:
spd = cds_contract.par_spread(settle_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  309.40594 bp


In [13]:
v = cds_contract.value(settle_date, issuer_curve, recovery_rate)

In [14]:
dirty_pv = v['dirty_pv'] 
clean_pv = v['clean_pv']

In [15]:
print("DIRTY VALUE %12.2f"% dirty_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

DIRTY VALUE   -115756.79
CLEAN VALUE   -117840.12


In [16]:
cleanp = cds_contract.clean_price(settle_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE    88.215988


In [17]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 50.0


In [18]:
accrued_interest = cds_contract.accrued_interest()
print("ACCRUED_COUPON", accrued_interest)

ACCRUED_COUPON 2083.3333333333335


In [19]:
prot_pv = cds_contract.protection_leg_pv(settle_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 228726.9467424124


In [20]:
premPV = cds_contract.premium_leg_pv(settle_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 112970.15765938828


In [21]:
rpv01 = cds_contract.risky_pv01(settle_date, issuer_curve)
print("DIRTY_RPV01", rpv01['dirty_rpv01'])
print("CLEAN_RPV01", rpv01['clean_rpv01'])

DIRTY_RPV01 7.531343843959219
CLEAN_RPV01 7.392454955070329


In [22]:
cds_contract.risky_pv01(settle_date, issuer_curve)

{'dirty_rpv01': 7.531343843959219, 'clean_rpv01': 7.392454955070329}

## Risk Measures

In [23]:
credit_dv01 = cds_contract.credit_dv01(settle_date, issuer_curve, recovery_rate)

In [24]:
credit_dv01

-650.5636208889919

Copyright (c) 2020 Dominic O'Kane